In [1]:
import os
import shutil
import sys
import numpy as np
from scipy import ndimage
import scipy
import matplotlib.pyplot as plt
import h5py
import tensorflow as tf

S:\Softwares\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_path = "S:\\Work\\Github_repos\\Fruits_Classification\\data\\fruits-360"

In [3]:
def read(data_path, folder_type='train'):
    if folder_type.lower() == 'train':
        path = data_path + "\\" + "Training"
    elif folder_type.lower() == 'test':
        path = data_path + "\\" + "Test"
    else:
        print("Wrong folder path")
        sys.exit(1)
    
    total_fruits_list  = os.listdir(path)
    total_fruits_count = len(total_fruits_list)
    
    
    
    all_images_count = 0
    
    for fruit in total_fruits_list:        
        for img in os.listdir(path + "\\" + fruit):
            all_images_count += 1
            
    
    data = {}
    labels = {}
    data[folder_type] = np.zeros(shape=(all_images_count, 100, 100, 3), dtype=np.float64)
    labels[folder_type] = []
    print(data[folder_type].shape)
    
    data_counter = 0
    label_counter = 0
    for fruit in total_fruits_list:        
        for img in os.listdir(path + "\\" + fruit):
            cur_img = ndimage.imread(path + "\\" + fruit + "\\" + img)
            #print(cur_img.shape, path + "\\" + fruit + "\\" + img)
            data[folder_type][data_counter,:,:,:] = cur_img            
            data_counter += 1
        labels[folder_type].append(fruit)
        print(folder_type, " : ", fruit)
        
            
    return data[folder_type], np.asarray(labels[folder_type])
    

In [ ]:
training_data, training_labels = read(data_path, folder_type="train")
test_data, test_labels = read(data_path, folder_type="test")

hf_train_data = h5py.File('training_data.h5', 'w')
hf_train_labels = h5py.File('training_labels.h5', 'w')
hf_test_data = h5py.File('test_data.h5', 'w')
hf_test_labels = h5py.File('test_labels.h5', 'w')

hf_train_data.create_dataset('training_data', data=training_data)
asciiList = [n.encode("ascii", "ignore") for n in training_labels]
hf_train_labels.create_dataset('training_labels', (len(asciiList),1),'S25', asciiList)

hf_test_data.create_dataset('test_data', data=test_data)
asciiList_test = [n.encode("ascii", "ignore") for n in test_labels]
hf_test_labels.create_dataset('test_labels', (len(asciiList_test),1),'S25', asciiList_test)

